In [1]:
import os
import pandas as pd

In [2]:
# Figure out path to pwd and data dir
_pwd = os.path.abspath("..")
_data_dir = os.path.join(_pwd, "data")

In [3]:
# Point your csv file here.
# Data can be collected from https://github.com/serval-snt-uni-lu/Piggybacking/blob/master/ground-truth/all-pairs.csv
_apk_csv = os.path.join(_data_dir, 'piggyback-all-pairs.csv')

In [4]:
API_KEY=open(os.path.join(_pwd, 'ANDROZOO_API_KEY.txt'),"r").read()
print(API_KEY)

e03b3548509b2b94d502314ca0f5cf7012c0357b18e6bea07071dec729c00c25


In [5]:
# Read the CSV
apk_df = pd.read_csv(_apk_csv)

In [6]:
# Find number of malicious apks for each benign apk.
apk_df['ORIGINAL_APP'].value_counts()

B45EB1CB7A931D963970F96150675F73926A3140D5376FF7A355FB07F4F55992    56
CE81D0A16CDCA3C6CCEB9A6607A47DC9DEBB7B7C9B84638263580F43427D4DFF    56
1DAF15DFCE13770C5CA76FAE2E1EE93EF62E56A267A6BE87D27C088FA9790870     3
A932CB38A67021F65A4FBA882E5B25FFA584036CD8BC617307E59F73F1398C63     3
CCC96109AB08504F4D1555A2B0F79AEB61D500463696E28159845FF65686AE03     3
                                                                    ..
81BEF3408E9036E4CAD808BDF2D522C8AD50F8C6CAC2E0FE3EDEA2D590D66A76     1
B69DB1ECD30380C70F7BDFF3E5B2DB845277653AC949B01D261DA2C78A3510C6     1
F5A2A7799AD591E114F34F494A234CE3516B46936DC63AE52EF8A8B4A0E7890C     1
3A29739071B89C4DAA188000AA2F63D8378B33AF6378B74A5CFB73AC542C5508     1
670BD04500567720057DDC07C8B990B4FA791CDAAE2EA476539F51CE15474A06     1
Name: ORIGINAL_APP, Length: 1355, dtype: int64

In [7]:
# Request the AndroZoo API for the apk and write it to a directory
# 
# params: 
#     api_key: Private key to the AndroZoo API
#     sha256: sha256 of the apk 
#    data_dir: directory to save the file. 
def request_apk_andro(api_key, sha256, data_dir):
    import requests
    url = "https://androzoo.uni.lu/api/download?"

    packaged_url = url + 'apikey=' + api_key + '&sha256=' + sha256
    print("Requesting", packaged_url)
    r = requests.get(packaged_url)
    
    filename = os.path.join(data_dir, sha256 + ".apk")
    with open(filename, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=128):
            fd.write(chunk)

In [12]:
# Gets all malicious apks for a corresponding benign apk. 
# The benign apks are stored in 
# 
# params: 
#     df: DataFrame with columns "ORIGINAL_APP" and "PIGGYBACKED_APP"
#     b_sha256: benign apk's sha256
#     data_dir: directory to save the file. 
def get_all_malicious(df, b_sha256, data_dir):
    malicious_sha_list = df.loc[df['ORIGINAL_APP'] == b_sha256]['PIGGYBACKED_APP']
    
    benign_folder_path = os.path.join(data_dir, "benign")
    print(benign_folder_path)

    if not os.path.isdir(benign_folder_path):
        os.makedirs(benign_folder_path)

    request_apk_andro(API_KEY, str(b_sha256), benign_folder_path)
    
    malicious_folder_path = os.path.join(data_dir, "malicious")
    print(malicious_folder_path)
    if not os.path.isdir(malicious_folder_path):
        os.makedirs(malicious_folder_path)
    
    for malicious_sha256 in malicious_sha_list:
        request_apk_andro(API_KEY, str(malicious_sha256), malicious_folder_path)

In [ ]:
for idx, sha256 in enumerate(apk_df['ORIGINAL_APP'].unique().tolist()[:1]):
    get_all_malicious(apk_df, sha256, os.path.join(_data_dir, f"app_{idx}"))

/Users/jarus/Work/r2callgraph4apk/data/app_0/benign
Requesting https://androzoo.uni.lu/api/download?apikey=e03b3548509b2b94d502314ca0f5cf7012c0357b18e6bea07071dec729c00c25&sha256=00575DE650F78413C91E8E613ADF9099812F325AA9CF36C72E81C79B230F58F4
/Users/jarus/Work/r2callgraph4apk/data/app_0/malicious
Requesting https://androzoo.uni.lu/api/download?apikey=e03b3548509b2b94d502314ca0f5cf7012c0357b18e6bea07071dec729c00c25&sha256=A55EA5807203CE31470D1905ED30F9267AE3459809DDA25C4F339197DE6486FE
